In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from IPython.display import clear_output, display

# Import ddm functions
from openddm.data_handling import read_file, export_data
from openddm.processing import ddm
from openddm.fitting import compute_AB, compute_ISF, singleExp, genFit
from openddm.plotting import plotSingleExpFit, plotISF

In [ ]:
# Optional to show a progressbar for dask computations
from openddm.utils import dask_progressbar
dask_progressbar()

In [ ]:
data_file = "../data/test_dataset_drift.tif"
output_path = "../results/"

In [ ]:
# DEBUG: Remove previous results 
if os.path.exists(output_path):
    shutil.rmtree(output_path)

In [ ]:
ddmData = read_file(data_file, delayed=True, chunk_size=10)

In [ ]:
A, B = compute_AB(ddmData)

In [ ]:
tau_range = np.arange(1,50)
tau_step = 5
tau_split = np.round(np.array_split(tau_range, len(tau_range)/tau_step).astype(int))

# Create figure
fig,ax = plt.subplots(1,1)

for taus in tau_split:
    ddmMatrix = ddm(ddmData, taus)
    ddmMatrix = export_data(output_path, ddmMatrix, taus, data_file)
    isf = compute_ISF(ddmMatrix.data, A, B)
    q = len(isf)//2
    
    # Plot result
    clear_output(wait=True)
    taus = ddmMatrix["tau"]
    ax.cla()
    
    ax.plot(taus, isf[q], ls = 'None', marker = 'o')

    # Fit data with singleExp
    try:
        pExpFit, pExpErrs = genFit(isf[q], taus, 'singleExp')
        ax.plot(taus, singleExp(taus, *pExpFit))
    except:
        pass
    
    ax.set_xlim(0,tau_range[-1]+1)
    ax.set_ylim(-0.2,1)
    ax.set_xlabel('lag time')
    ax.set_ylabel('ISF')
    display(fig)
clear_output()

### Interactive visualization with ipywidgets

In [ ]:
%matplotlib widget
import ipywidgets as widgets

In [ ]:
# Reset and reload data
data_file = "../data/test_dataset_drift.tif"
output_path = "../results/"
ddmData = read_file(data_file, delayed=True, chunk_size=10)
A, B = compute_AB(ddmData)
ddmMatrix = xr.open_dataarray("../results/test_dataset_drift_matrix.nc")
isf = compute_ISF(ddmMatrix.data, A, B)
taus = ddmMatrix["tau"]

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=(6, 4))

@widgets.interact(q=(0, len(isf)))
def update(q=len(isf)//2):
    ax.cla()
    ax.plot(taus, isf[q], color="b", ls = 'None', marker = 'o')
    try:
        pExpFit, pExpErrs = genFit(isf[q], taus, 'singleExp')
        ax.plot(taus, singleExp(taus, *pExpFit), color="r")
    except:
        pass
    ax.set_xlabel('lag time')
    ax.set_ylabel('ISF')
    ax.set_xlim(0,taus[-1])
    ax.set_ylim(-0.2,1)
